In [40]:
import os
import time
import json
import math
import talib
import selenium
import datetime
import requests
import threading
import numpy as np
import pandas as pd
import telegram_bot
from alice_blue import *
from datetime import timedelta
from sleep_function import sleeper
from collections import namedtuple
from collections import defaultdict
from fyers_api import accessToken
from fyers_api import fyersModel
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from IPython.display import clear_output, display
from jugaad_trader import Zerodha

In [41]:
class AmadeusData_AliceBlue():
    
    username, password, twoFA, api_secret = pd.read_csv('alice_blue_credentials.csv').iloc[0]
    
    def __init__(self, timeframe, ins_exc):
        
        self.access_token = AliceBlue.login_and_get_access_token(username = self.username, 
                                                                 password = self.password, 
                                                                 twoFA = self.twoFA,  
                                                                 api_secret = self.api_secret)
        
        self.alice = AliceBlue(username = self.username, password = self.password, access_token = self.access_token)
    
        self.Instrument = namedtuple('Instrument', ['exchange', 'token', 'symbol',
                                      'name', 'expiry', 'lot_size'])
    
        self.socket_opened = False
        
        self.timeframe = timeframe

        self.raw_data = defaultdict(
            lambda: pd.DataFrame(columns = ['datetime', 'ticker', 'ltp', 'cum_volume', 'volume']).set_index('datetime'))

        self.data = defaultdict(
            lambda: pd.DataFrame(columns = ['datetime', 'ticker', 'open', 'high', 'low', 'close', 'volume', 'date', 'time']).set_index('datetime'))
        
        self.ins_exc = ins_exc
        
        self.instruments = list(self.ins_exc.keys())
        
    def get_live_data(self):
        
        self.start_live_feed()
        self.start_thread()
        
    def get_ltp(self, message):
    
        self.message = message
        
        self.instrument = self.message['instrument'].symbol
        
        if self.instrument not in self.instruments:
            self.instruments.append(self.instrument)

        self.timestamp = datetime.datetime.fromtimestamp(self.message['exchange_time_stamp'])
        
        self.raw_data[self.instrument] = self.raw_data[self.instrument].append(
                            pd.Series({'ticker' : self.instrument, 'ltp' : self.message['ltp'], 'cum_volume' : self.message['volume']}, 
                            name = self.timestamp))
        
        self.raw_data[self.instrument]['volume'] = self.raw_data[self.instrument]['cum_volume'].diff()
        
    def open_callback(self):
    
        self.socket_opened = True
        
    
    def start_live_feed(self):
    
        self.alice.start_websocket(subscribe_callback=self.get_ltp,
                          socket_open_callback=self.open_callback,
                          run_in_background=True)

        while(self.socket_opened==False):
            pass

        for key in self.ins_exc:
        
            self.alice.subscribe(self.alice.get_instrument_by_symbol(self.ins_exc[key]['exchange'], key), LiveFeedType.COMPACT)

            kite = Zerodha()
            kite.load_creds()
            kite.login()

            if (key[-3:] == 'FUT'):

                ins_kite = self.ins_exc[key]['exchange'] + ':' + key.split(' ')[0] + str(datetime.date.today().year)[2:] + key.split(' ')[1] + key.split(' ')[2]
                ins_kite_number = kite.quote(ins_kite)[ins_kite]['instrument_token']
                from_date = datetime.date.today() - datetime.timedelta(days = 5)
                to_date = datetime.date.today()
                interval = "minute"
                self.data[key] = pd.DataFrame(kite.historical_data(ins_kite_number, from_date, to_date, interval, continuous=False, oi=False))

            else:

                ins_kite = self.ins_exc[key]['exchange'] + ':' + key
                ins_kite_number = kite.quote(ins_kite)[ins_kite]['instrument_token']
                from_date = datetime.date.today() - datetime.timedelta(days = 5)
                to_date = datetime.date.today()
                interval = "minute"
                self.data[key] = pd.DataFrame(kite.historical_data(ins_kite_number, from_date, to_date, interval, continuous=False, oi=False))


            self.data[key]['datetime'] = self.data[key]['date']
            self.data[key]['ticker'] = key

            self.data[key]['date'] = self.data[key]['datetime'].apply(lambda x : x.date())
            self.data[key]['time'] = self.data[key]['datetime'].apply(lambda x : x.time())

            self.data[key] = self.data[key][['datetime','ticker','open','high','low','close','volume','date','time']]
            self.data[key]['datetime'] = self.data[key]['date'].apply(str) + self.data[key]['time'].apply(str)
            self.data[key]['datetime'] = self.data[key]['datetime'].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d%H:%M:%S'))
            self.data[key].set_index('datetime', inplace=True)
            
    @staticmethod
    def obtain_timedelta(timeframe):

        if timeframe[-1] == 'S':
            td = datetime.timedelta(seconds=int(timeframe[:-1]))

        elif timeframe[-1] == 'T':
            td = datetime.timedelta(minutes=int(timeframe[:-1]))

        elif timeframe[-3:].lower() == 'min':
            td = datetime.timedelta(minutes=int(timeframe[:-3]))

        elif timeframe[-1] == 'H':
            td = datetime.timedelta(hours=int(timeframe[:-1]))

        elif timeframe[-1] == 'D':
            td = datetime.timedelta(days=int(timeframe[:-1]))

        elif timeframe[-1] == 'W':
            td = datetime.timedelta(weeks=int(timeframe[:-1]))

        else:
            td = None

        return td
    
    def resample_data(self):
    
        self.thread_resample_run = True
        
        while self.thread_resample_run:

            for ins in self.instruments:
                
                if len(self.raw_data[ins]) == 0:

                    continue
                                                    
                temp = pd.DataFrame({'open' : self.raw_data[ins]['ltp'].resample(self.timeframe).first(),
                                     'high' : self.raw_data[ins]['ltp'].resample(self.timeframe).max(),
                                     'low' : self.raw_data[ins]['ltp'].resample(self.timeframe).min(),
                                     'close' : self.raw_data[ins]['ltp'].resample(self.timeframe).last(),
                                     'volume' : self.raw_data[ins]['volume'].resample(self.timeframe).sum()
                                    })

                temp['ticker'] = ins
                temp['date'] = temp.index.date
                temp['time'] = temp.index.time

                self.data[ins].at[temp.index[0]] = temp.iloc[0]
                
                if temp.index[-1] > self.data[ins].index[-1]:

                    previous_tick = self.data[ins].index[-1] + self.obtain_timedelta(self.timeframe)

                    self.raw_data[ins] = self.raw_data[ins][self.raw_data[ins].index > previous_tick]
    
    def start_thread(self):

        self.thread_resample = threading.Thread(target = self.resample_data)
        self.thread_resample.start()
        
    def terminate_thread(self):
            
        self.thread_resample_run = False
        self.thread_resample.join() 


In [42]:
class GamePlan:
    
    def __init__(self, instrument, exchange, quantity):

        self.instrument = instrument
        self.percentage_padding = 0.02
        self.capital = 0
        self.position_size = quantity
        self.exchange = exchange

        if amadeus_data.data[self.instrument].iloc[-1]['close'] < 500:
            self.stoploss = round(amadeus_data.data[self.instrument].iloc[-1]['close']/500, 1)
            
        elif self.instrument[:5] == 'NIFTY':
            self.stoploss = 10
            
        elif self.instrument[:9] == 'BANKNIFTY':
            self.stoploss = 40
        
        else:
            self.stoploss = round(amadeus_data.data[self.instrument].iloc[-1]['close']/500)

        self.today = pd.Timestamp(datetime.date.today())
        
        if self.exchange == 'MCX':
            self.market_close = datetime.time(hour=23, minute=30)
        else:
            self.market_close = datetime.time(hour=15, minute=30)
        
        self.df = amadeus_data.data[self.instrument].copy()
        self.prices_calculated_flag = False
        self.long_position_flag = False
        self.long_target_1_flag = False
        self.long_target_2_flag = False
        self.short_position_flag = False
        self.short_target_1_flag = False
        self.short_target_2_flag = False
        
    @staticmethod
    def price_range(current_price, price_reference, percentage):

        return ((price_reference * (1 - percentage/100) < current_price) & 
                    (current_price < price_reference * (1 + percentage/100)))
    
    def send_signals(self):

        self.thread_send_signals = threading.Thread(target = self.send_signals_for_thread)
        self.thread_send_signals.start()
        
    def terminate_signals(self):
            
        self.signals_run_thread_flag = False
 
    def create_price_levels(self):
        
        self.index = amadeus_data.data[self.instrument].index[-1]
        self.row = amadeus_data.data[self.instrument].iloc[-1]

        if ((self.index.date() == self.today) & 
           (self.index.time() > datetime.time(hour=9, minute=30)) & 
           (self.prices_calculated_flag == False)):

            self.candle = amadeus_data.data[self.instrument][(amadeus_data.data[self.instrument]['time'] >= datetime.time(hour=9, minute=15)) & 
                                                              (amadeus_data.data[self.instrument]['time'] <= datetime.time(hour=9, minute=30)) &
                                                              (amadeus_data.data[self.instrument]['date'] == self.today)]

            self.high = self.candle['high'].max()
            self.low = self.candle['low'].min()

            self.diff = round(self.high - self.low, 3)
            self.atp = round((self.high + self.low) / 2, 3)

            self.derived = round(self.diff * 0.618, 3)

            self.buy = round(self.atp + self.stoploss, 3)
            self.sell = round(self.atp - self.stoploss, 3)

            self.buy_momentum = round(self.atp + self.derived, 3)
            self.sell_momentum = round(self.atp - self.derived, 3)

            self.long_target_1 = round(self.buy_momentum + self.diff, 3)
            self.long_target_2 = round(self.long_target_1 + self.diff, 3)

            self.short_target_1 = round(self.sell_momentum - self.diff, 3)
            self.short_target_2 = round(self.short_target_1 - self.diff, 3)

            self.price_levels = {
                                    'stoploss' : self.stoploss,
                                    'high': self.high,
                                    'low': self.low,
                                    'buy': self.buy,
                                    'sell': self.sell,
                                    'buy_momentum': self.buy_momentum,
                                    'sell_momentum': self.sell_momentum,
                                    'long_target_1': self.long_target_1,
                                    'long_target_2': self.long_target_2,
                                    'short_target_1': self.short_target_1,
                                    'short_target_2': self.short_target_2
                                }

            self.prices_calculated_flag = True
 
    def send_signals_for_thread(self):

        self.signals_run_thread_flag = True
        
        while self.signals_run_thread_flag:
            
            self.index = amadeus_data.data[self.instrument].index[-1]
            self.row = amadeus_data.data[self.instrument].iloc[-1]
            
            if ((self.index.date() == self.today) &
                (self.index.time() > datetime.time(hour=9, minute=30))):
                
                if self.prices_calculated_flag == False:
                    self.create_price_levels()

                self.df.loc[self.index] = self.row
                self.df['cci'] = talib.CCI(self.df['high'],
                                      self.df['low'],
                                      self.df['close'],
                                      timeperiod=369)

                self.ltp = self.df.iloc[-1]['close']
                self.cci = self.df.iloc[-1]['cci']
                
                self.price_flags = {
                                        'long_position_flag': self.long_position_flag,
                                        'long_target_1_flag': self.long_target_1_flag,
                                        'long_target_2_flag': self.long_target_2_flag,
                                        'short_position_flag': self.short_position_flag,
                                        'short_target_1_flag': self.short_target_1_flag,
                                        'short_target_2_flag': self.short_target_2_flag
                                    }

                if (self.long_position_flag | self.short_position_flag) == False:
   
                    
                    if self.price_range(self.ltp, self.buy, self.percentage_padding) & (self.cci > 20):

                        self.long_position_flag = True
                        self.long_target_1_flag = False
                        self.long_target_2_flag = False
                        self.capital = self.capital - (self.position_size * self.ltp)
                        telegram_bot.send(f'{self.instrument}\n{self.index}\nLONG ENTRY - Price : {self.ltp} \nGameplan Buy Price : {self.buy}')
                        print(f'{self.instrument}\n{self.index}\nLONG ENTRY - Price : {self.ltp} \nGameplan Buy Price : {self.buy}')
                        print('______________________________________________')

                    elif self.price_range(self.ltp, self.sell, self.percentage_padding) & (self.cci < -20):

                        self.short_position_flag = True
                        self.short_target_1_flag = False
                        self.short_target_2_flag = False
                        self.capital = self.capital + (self.position_size * self.ltp)
                        telegram_bot.send(f'{self.instrument}\n{self.index}\nSHORT ENTRY - Price : {self.ltp} \nGameplan Sell Price : {self.sell}')
                        print(f'{self.instrument}\n{self.index}\nSHORT ENTRY - Price : {self.ltp} \nGameplan Sell Price : {self.sell}')
                        print('______________________________________________')

                if self.long_position_flag:
                    
                    
                    if (self.ltp > self.long_target_1) & (self.long_target_1_flag == False):

                        self.long_target_1_flag = True
                        self.capital = self.capital + (self.position_size/2 * self.ltp)
                        telegram_bot.send(f'{self.instrument}\n{self.index}\nHIT LONG TARGET 1 - Price : {self.ltp} \nGameplan Target 1 Price : {self.long_target_1}')
                        print(f'{self.instrument}\n{self.index}\nHIT LONG TARGET 1 - Price : {self.ltp} \nGameplan Target 1 Price : {self.long_target_1}')
                        print('______________________________________________')

                    elif (self.ltp > self.long_target_2) & (self.long_target_2_flag == False) & (self.long_target_1_flag == True):

                        self.long_target_2_flag = True
                        self.long_position_flag = False
                        self.long_target_1_flag = False
                        self.long_target_2_flag = False
                        self.short_position_flag = False
                        self.short_target_1_flag = False
                        self.short_target_2_flag = False
                        self.capital = self.capital + (self.position_size/2 * self.ltp)
                        telegram_bot.send(f'{self.instrument}\n{self.index}\nHIT LONG TARGET 2 - Price : {self.ltp} \nGameplan Target 2 Price : {self.long_target_2}')
                        print(f'{self.instrument}\n{self.index}\nHIT LONG TARGET 2 - Price : {self.ltp} \nGameplan Target 2 Price : {self.long_target_2} \nCapital : {self.capital}')
                        print('______________________________________________')

                    elif (self.ltp < self.buy_momentum) & (self.long_target_1_flag == True) & (self.long_target_2_flag == False):

                        self.long_position_flag = False
                        self.long_target_1_flag = False
                        self.long_target_2_flag = False
                        self.short_position_flag = False
                        self.short_target_1_flag = False
                        self.short_target_2_flag = False
                        self.capital = self.capital + (self.position_size/2 * self.ltp)
                        telegram_bot.send(f'{self.instrument}\n{self.index}\nHIT STOP LOSS AFTER Target 1 - Price : {self.ltp} \nGameplan Buy Momentum : {self.buy_momentum}')
                        print(f'{self.instrument}\n{self.index}\nHIT STOP LOSS AFTER Target 1 - Price : {self.ltp} \nGameplan Buy Momentum : {self.buy_momentum}')
                        print('______________________________________________')

                    elif self.ltp < self.sell:

                        if self.long_target_1_flag:

                            self.long_position_flag = False
                            self.long_target_1_flag = False
                            self.long_target_2_flag = False
                            self.short_position_flag = False
                            self.short_target_1_flag = False
                            self.short_target_2_flag = False
                            self.capital = self.capital + (self.position_size/2 * self.ltp)
                            telegram_bot.send(f'{self.instrument}\n{self.index}\nHIT STOP LOSS - Price : {self.ltp} \nGameplan Sell Price : {self.sell}')
                            print(f'{self.instrument}\n{self.index}\n HIT STOP LOSS - Price : {self.ltp} \nGameplan Sell Price : {self.sell}')
                            print('______________________________________________')

                        else:

                            self.long_position_flag = False
                            self.long_target_1_flag = False
                            self.long_target_2_flag = False
                            self.short_position_flag = False
                            self.short_target_1_flag = False
                            self.short_target_2_flag = False
                            self.capital = self.capital + (self.position_size * self.ltp)
                            telegram_bot.send(f'{self.instrument}\n{self.index}\nHIT STOP LOSS - Price : {self.ltp} \nGameplan Sell Price : {self.sell}')
                            print(f'{self.instrument}\n{self.index}\nHIT STOP LOSS - Price : {self.ltp} \nGameplan Sell Price : {self.sell} \nCapital : {self.capital}')
                            print('______________________________________________')

                        if self.cci < -20:

                            self.short_position_flag = True
                            self.capital = self.capital + (self.position_size * self.ltp)
                            telegram_bot.send(f'{self.instrument}\n{self.index}\nHIT SHORT ENTRY - Price : {self.ltp} \nGameplan Sell Price : {self.sell}')
                            print(f'{self.instrument}\n{self.index}\nHIT SHORT ENTRY - Price : {self.ltp} \nGameplan Buy Price : {self.sell}')
                            print('______________________________________________')

                if self.short_position_flag:                    

                    if (self.ltp < self.short_target_1) & (self.short_target_1_flag == False):

                        self.short_target_1_flag = True
                        self.capital = self.capital - (self.position_size/2 * self.ltp)
                        telegram_bot.send(f'{self.instrument}\n{self.index}\nHIT SHORT TARGET 1 - Price : {self.ltp} \nGameplan Target 1 Price : {self.short_target_1}')
                        print(f'{self.instrument}\n{self.index}\nHIT SHORT TARGET 1 - Price : {self.ltp} \nGameplan Target 1 Price : {self.short_target_1}')
                        print('______________________________________________')

                    elif (self.ltp < self.short_target_2) & (self.short_target_2_flag == False) & (self.short_target_1_flag == True):

                        self.short_target_2_flag = True
                        self.long_position_flag = False
                        self.long_target_1_flag = False
                        self.long_target_2_flag = False
                        self.short_position_flag = False
                        self.short_target_1_flag = False
                        self.short_target_2_flag = False
                        self.capital = self.capital - (self.position_size/2 * self.ltp)
                        telegram_bot.send(f'{self.instrument}\n{self.index}\nHIT SHORT TARGET 2 - Price : {self.ltp} \nGameplan Target 2 Price : {self.short_target_2}')
                        print(f'{self.instrument}\n{self.index}\nHIT SHORT TARGET 2 - Price : {self.ltp} \nGameplan Target 2 Price : {self.short_target_2} \nCapital : {self.capital}')
                        print('______________________________________________')

                    elif (self.ltp > self.sell_momentum) & (self.short_target_1_flag == True) & (self.short_target_2_flag == False):

                        self.long_position_flag = False
                        self.long_target_1_flag = False
                        self.long_target_2_flag = False
                        self.short_position_flag = False
                        self.short_target_1_flag = False
                        self.short_target_2_flag = False
                        self.capital = self.capital - (self.position_size/2 * self.ltp)
                        telegram_bot.send(f'{self.instrument}\n{self.index}\nHIT STOP LOSS - Price : {self.ltp} \nGameplan Buy Price : {self.sell_momentum}')
                        print(f'{self.instrument}\n{self.index}\nHIT STOP LOSS - Price : {self.ltp} \nGameplan Buy Price : {self.sell_momentum}')
                        print('______________________________________________')

                    elif self.ltp > self.buy:

                        if self.short_target_1_flag:

                            self.long_position_flag = False
                            self.long_target_1_flag = False
                            self.long_target_2_flag = False
                            self.short_position_flag = False
                            self.short_target_1_flag = False
                            self.short_target_2_flag = False
                            self.capital = self.capital - (self.position_size/2 * self.ltp)
                            telegram_bot.send(f'{self.instrument}\n{self.index}\nHIT STOP LOSS - Price : {self.ltp} \nGameplan Buy Price : {self.buy}')
                            print(f'{self.instrument}\n{self.index}\nHIT STOP LOSS - Price : {self.ltp} \nGameplan Buy Price : {self.buy} \nCapital : {self.capital}')
                            print('______________________________________________')

                        else:

                            self.long_position_flag = False
                            self.long_target_1_flag = False
                            self.long_target_2_flag = False
                            self.short_position_flag = False
                            self.short_target_1_flag = False
                            self.short_target_2_flag = False
                            self.capital = self.capital - (self.position_size * self.ltp)
                            telegram_bot.send(f'{self.instrument}\n{self.index}\nHIT STOP LOSS - Price : {self.ltp} \nGameplan Sell Price : {self.buy}')
                            print(f'{self.instrument}\n{self.index}\nHIT STOP LOSS - Price : {self.ltp} \nGameplan Buy Price : {self.buy} \nCapital : {self.capital}')
                            print('______________________________________________')

                        if self.cci > 20:

                            self.long_position_flag = True
                            self.capital = self.capital - (self.position_size * self.ltp)
                            telegram_bot.send(f'{self.instrument}\n{self.index}\nLONG ENTRY - Price : {self.ltp} \nGameplan Buy Price : {self.buy}')
                            print(f'{self.instrument}\n{self.index}\nHIT LONG ENTRY - Price : {self.ltp} \nGameplan Buy Price : {self.buy}')
                            print('______________________________________________')


                if (((self.long_position_flag | self.short_position_flag) == True) & (self.index.time() >= datetime.time(hour=self.market_close.hour, minute=self.market_close.minute-5))):

                    if self.long_position_flag:

                        if self.long_target_1_flag:

                            self.long_position_flag = False
                            self.long_target_1_flag = False
                            self.long_target_2_flag = False
                            self.short_position_flag = False
                            self.short_target_1_flag = False
                            self.short_target_2_flag = False
                            self.capital = self.capital + (self.position_size/2 * self.ltp)
                            telegram_bot.send(f'{self.instrument}\n{self.index}\nEND OF DAY CLOSING LONG POSITION - Price : {self.ltp}')
                            print(f'{self.instrument}\n{self.index}\nEND OF DAY CLOSING LONG POSITION - Price : {self.ltp} \nCapital : {self.capital}')
                            print('______________________________________________')

                        else:

                            self.long_position_flag = False
                            self.long_target_1_flag = False
                            self.long_target_2_flag = False
                            self.short_position_flag = False
                            self.short_target_1_flag = False
                            self.short_target_2_flag = False
                            self.capital = self.capital + (self.position_size * self.ltp)
                            telegram_bot.send(f'{self.instrument}\n{self.index}\nEND OF DAY CLOSING LONG POSITION - Price : {self.ltp}')
                            print(f'{self.instrument}\n{self.index}\nEND OF DAY CLOSING LONG POSITION - Price : {self.ltp} \nCapital : {self.capital}')
                            print('______________________________________________')


                    if self.short_position_flag:

                        if self.short_target_1_flag:

                            self.long_position_flag = False
                            self.long_target_1_flag = False
                            self.long_target_2_flag = False
                            self.short_position_flag = False
                            self.short_target_1_flag = False
                            self.short_target_2_flag = False
                            self.capital = self.capital - (self.position_size/2 * self.ltp)
                            telegram_bot.send(f'{self.instrument}\n{self.index}\nEND OF DAY CLOSING SHORT POSITION- Price : {self.ltp}')
                            print(f'{self.instrument}\n{self.index}\nEND OF DAY CLOSING SHORT POSITION - Price : {self.ltp} \nCapital : {self.capital}')
                            print('______________________________________________')

                        else:

                            self.long_position_flag = False
                            self.long_target_1_flag = False
                            self.long_target_2_flag = False
                            self.short_position_flag = False
                            self.short_target_1_flag = False
                            self.short_target_2_flag = False
                            self.capital = self.capital - (self.position_size * ltp)
                            telegram_bot.send(f'{self.instrument}\n{self.index}\nEND OF DAY CLOSING SHORT POSITION- Price : {self.ltp}')
                            print(f'{self.instrument}\n{self.index}\nEND OF DAY CLOSING SHORT POSITION - Price : {self.ltp} \nCapital : {self.capital}')
                            print('______________________________________________')
                            
                    self.signals_run_thread_flag = False
                

In [ ]:
if __name__ == '__main__':
    
    ins_exc = {
                    'AXISBANK AUG FUT': {'exchange' : 'NFO', 'quantity' : 2},
                    'BAJFINANCE AUG FUT': {'exchange' : 'NFO', 'quantity' : 2},
                    'ICICIBANK AUG FUT': {'exchange' : 'NFO', 'quantity' : 2},
                    'NIFTY AUG FUT': {'exchange' : 'NFO', 'quantity' : 2},
                    'BANKNIFTY AUG FUT': {'exchange' : 'NFO', 'quantity' : 2}
              }
    
#     ins_exc = {
#                     'GOLDM AUG FUT': {'exchange' : 'MCX', 'quantity' : 2},
#               }
    
    amadeus_data = AmadeusData_AliceBlue('1min', ins_exc)
    
#     sleeper(9, 20)
    
    amadeus_data.get_live_data()

In [ ]:
amadeus_data.raw_data['AXISBANK AUG FUT']

In [ ]:
game_plan = {}
    
for instrument in ins_exc:

    game_plan[instrument] = GamePlan(instrument, ins_exc[instrument]['exchange'], ins_exc[instrument]['quantity'])
    game_plan[instrument].send_signals()

In [ ]:
game_plan['AXISBANK AUG FUT'].index.time() >= datetime.time(
    hour=game_plan['AXISBANK AUG FUT'].market_close.hour, minute=game_plan['AXISBANK AUG FUT'].market_close.minute - 5)

In [ ]:
game_plan['AXISBANK AUG FUT'].ltp

In [ ]:
game_plan['AXISBANK AUG FUT'].price_flags

***